In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import *

#TODO actually load real weights from a generated model
class ALSModel(nn.Module):
    def __init__(self, n_items: int, n_factors: int):
        super(ALSModel, self).__init__()
        #TODO add user --> item
        self.emb = nn.Embedding(n_items, n_factors)
        self.it2ind = {str(i): i for i in range(n_items)}# TODO use real values
        self.ind2it = {v:k for k,v in self.it2ind.items()}
        
    
    def map_out(self, scores: torch.Tensor, indices: torch.Tensor) -> Dict[str, float]:
        inds: List[int] = indices.squeeze().tolist()
        scorez: List[float] = scores.squeeze().tolist()
        return {self.ind2it[i]: s for i, s in zip(inds, scorez)}
    
    #def forward(self, ind: torch.Tensor, size: int = 30)-> torch.Tensor:#  -> Dict[str, float]:
    def forward(self, ind: torch.Tensor)-> torch.Tensor:
    #def forward(self, ind: torch.Tensor, size: int = 5)-> torch.Tensor:
        #ind = torch.tensor([self.it2ind[inp]]).long()
        u = self.emb(ind)
        scores = u @ self.emb.weight.t()
        s, i = scores.topk(5)
        #return self.map_out(s, i)
        return s.squeeze()
        
        
n_items, n_factors = (10000, 128)     
m = ALSModel(n_items, n_factors)
m2 = torch.jit.script(m)
    
#TODO can we actually map strings and stuff in ONNX? It seems like we cant??? We cant even send in ints to modify functions.
#TODO export model to ONNX
#m("5", size=5), m2("5", size=5)


In [8]:
m2(torch.ones(1).long())

tensor([107.4750,  46.6246,  41.8947,  38.1183,  36.3196],
       grad_fn=<SqueezeBackward0>)

In [9]:
x = (torch.ones(1).long())
#x = (torch.ones(1).long(), 3)
file_path = "data/model.onnx"
torch.onnx.export(m2, x, file_path, verbose=True)

graph(%ind.1 : Long(1, strides=[1], requires_grad=0, device=cpu),
      %emb.weight : Float(10000, 128, strides=[128, 1], requires_grad=0, device=cpu),
      %8 : Float(128, 10000, strides=[1, 128], requires_grad=0, device=cpu)):
  %u : Float(1, 128, strides=[128, 1], device=cpu) = onnx::Gather(%emb.weight, %ind.1) # /home/n651042/micromamba/envs/attention/lib/python3.10/site-packages/torch/nn/functional.py:2183:11
  %scores : Float(1, 10000, strides=[10000, 1], device=cpu) = onnx::MatMul(%u, %8) # /tmp/ipykernel_12857/3742737406.py:26:17
  %s : Float(1, 5, strides=[5, 1], device=cpu), %i : Long(1, 5, strides=[5, 1], device=cpu) = onnx::TopK[axis=-1, k=5](%scores) # /tmp/ipykernel_12857/3742737406.py:27:15
  %7 : Float(5, strides=[1], requires_grad=1, device=cpu) = onnx::Squeeze(%s) # /tmp/ipykernel_12857/3742737406.py:29:15
  return (%7)



In [10]:
import onnx
# ensure a valid onnx file
onnx.checker.check_model(onnx.load(file_path))

In [11]:
import onnxruntime
import numpy as np

# sanity test outputs
ort_session = onnxruntime.InferenceSession(file_path)
inp = np.ones(1, dtype=np.int64) * 99
ort_inputs = {ort_session.get_inputs()[0].name: inp}
ort_outs = ort_session.run(None, ort_inputs)

print(ort_outs)

[array([149.96411 ,  44.02415 ,  43.7658  ,  41.755444,  41.204   ],
      dtype=float32)]
